In [1]:
# import all the packages

from bs4 import BeautifulSoup
import urllib.request as ul
import csv
import sqlite3

In [15]:
# function that scrap an entire html page

def page_table(tag, table):
    print(tag)
    # url of the page number 'tag'
    urlpage = 'https://books.toscrape.com/catalogue/page-'+ tag +'.html'

    page = ul.urlopen(urlpage)
    # html file in the soup variable
    soup = BeautifulSoup(page, 'lxml')
    # ResultSet (list) of <article> tags
    articles = soup.find_all('article')
    
    # loop filling the table
    for j in range(len(articles)):
        # empty line to append at the end
        line_ = []
        # title of the book in the <a> title
        line_.append(articles[j].h3.a['title'])
        # getting the categorical rating (stars)
        rating = articles[j].p['class'][1]
        # replacing the string into a integer corresponding
        rating_str = rating.replace('One', '1')
        rating_str = rating_str.replace('Two', '2')
        rating_str = rating_str.replace('Three', '3')
        rating_str = rating_str.replace('Four', '4')
        rating_str = rating_str.replace('Five', '5')
        line_.append(int(rating_str))
        # getting a <div> with the 'product_price' name
        div_prod = articles[j].find('div', {'class': 'product_price'})
        # getting the cleaned price of the book
        line_.append(div_prod.p.text[1:])
        # getting the cleaned link of the image
        src = articles[j].div.img['src'].strip('..')
        line_.append('https://books.toscrape.com' + src)
        # getting the availibility of the book (turned into a 0 or 1)
        line_.append(int(div_prod.p.find_next('p')['class'][0] == 'instock'))
        # adding the entire line to the table
        table.append(line_)
    return table

In [13]:
# calling the function on multiple pages
def scraping_to_table():
    # initilalize global table
    table = []
    # getting the number of pages of the site
    urlpage = 'https://books.toscrape.com/'
    page = ul.urlopen(urlpage)
    soup = BeautifulSoup(page, 'html.parser')
    nbrpages = int(soup.find(class_="current").text.strip().split()[3])
    # calling the function for all 
    for i in range(1, nbrpages+1):
        table = page_table(str(i), table)
    return table

In [14]:
# function called in the main
def scraping_to_csv():
    # calling the scraping function
    table = scraping_to_table()
    # columns of the table
    columns = ['title', 'rank', 'price','link', 'availibility']
    # opening and writing of the csv file
    with open('books.csv', 'w', newline = '') as f:
        write = csv.writer(f)
        write.writerow(columns)
        write.writerows(table)
    # the function creates a csv file on the root of the script
    return